<a href="https://colab.research.google.com/github/sujeongEOM/CXR-BDM/blob/main/%5BCXR_BDM%5D_Chest_PA_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
pip install pydicom

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pydicom as dicom
import os
import pickle

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Endo-CXR/chestpa_BMD_20210414DXA.xls - Sheet1.csv')

In [ ]:
basic_path = '/content/drive/MyDrive/Endo-CXR'
both = '/content/drive/MyDrive/Endo-CXR/Chest PA, Lat (Both)'
lt = '/content/drive/MyDrive/Endo-CXR/Chest PA, Lat (Lt)'
rt = '/content/drive/MyDrive/Endo-CXR/Chest PA, Lat (Rt)'

# PA - designated

In [ ]:
#pa파일을 직접 지정한 pt id와 filename
DesigID = [...]
DesigPA = [...] #직접 출력해주는 케이스들


#PA - using header metadata

In [ ]:
HeaderID = []
HeaderPA = []

for (path, dir, files) in os.walk(basic_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            file_path = "%s/%s" % (path, filename)
            ds = dicom.dcmread(file_path)
            if ds.PatientID in DesigID:
                pass  
            else:
                try:
                    #series description으로 pa 추출
                    if ds.SeriesDescription == 'CHEST PA' or ds.SeriesDescription == 'PA' or ds.SeriesDescription == 'AP' or ds.SeriesDescription == 'CHEST AP':
                        HeaderID.append(ds.PatientID)
                        HeaderPA.append(file_path)
                    #series description으로 안 되는 경우
                    elif ds.SeriesDescription == 'Chest' or ds.SeriesDescription == 'CHEST':
                        if ds.ViewPosition == 'PA' or ds.ViewPosition == 'AP':
                            HeaderID.append(ds.PatientID)
                            HeaderPA.append(file_path)
                        elif ds.AcquisitionDeviceProcessingDescription == 'CHEST - PA' or ds.AcquisitionDeviceProcessingDescription == 'CHEST PA':
                            HeaderID.append(ds.PatientID)
                            HeaderPA.append(file_path)
                        elif ds.ProtocolName == 'CHEST PA(LDI)':
                            HeaderID.append(ds.PatientID)
                            HeaderPA.append(file_path)
                except AttributeError as e:
                    if ds.AcquisitionDeviceProcessingDescription == 'CHEST - PA' or ds.AcquisitionDeviceProcessingDescription == 'CHEST PA':
                        HeaderID.append(ds.PatientID)
                        HeaderPA.append(file_path)


In [ ]:
# save
with open(basic_path + '/HeaderID.pickle', 'wb') as f:
    pickle.dump(HeaderID, f)

with open(basic_path + '/HeaderPA.pickle', 'wb') as f:
    pickle.dump(HeaderPA, f)

In [ ]:
#load
with open(basic_path + '/HeaderID.pickle', 'rb') as f:
    HeaderID = pickle.load(f)

with open(basic_path + '/HeaderPA.pickle', 'rb') as f:
    HeaderPA = pickle.load(f)

#PA - first file in directory

In [ ]:
# dcm data로 pa 출력 안 돼서 환자 폴더 내 첫번째 파일 출력하기 (보통 첫번째 파일이 pa)
# x-ray 직접 추출 시 폴더 명을 pt id로 했음

TotalID = set(os.listdir(both) + os.listdir(rt) + os.listdir(lt))
FirstLocID = TotalID - set(DesigID) - set(HeaderID)
FirstLocPA = []


for i in FirstLocID:
    f_list = []
    if i in os.listdir(both):
        path = both + "/" + i
    elif i in os.listdir(lt):
        path = lt + "/" + i
    elif i in os.listdir(rt):
        path = rt + "/" + i
#lat(lt) or lat(rt)는 파일 개수가 2개여서 가능한 방식    
    f_filenames = os.listdir(path)
    if f_filenames[0] > f_filenames[1]:
        pa_file = path + "/" + f_filenames[1]
    else:
        pa_file = path + "/" + f_filenames[0] 
    FirstLocPA.append(pa_file)

#PA - total file_paths

In [ ]:
TotalPA = DesigPA + HeaderPA + FirstLocPA

In [ ]:
# save
with open(basic_path + '/TotalPA', 'wb') as f:
    pickle.dump(TotalPA, f)

In [ ]:
#load
with open(basic_path + '/TotalPA', 'rb') as f:
    TotalPA = pickle.load(f)